In [26]:
import torch
import torch.nn.functional as F
from sophia import SophiaG
import torch.nn as nn
from utils import read_data, default_dataset_parameters
from torch.utils.data import TensorDataset, DataLoader


In [31]:
data_name = 'a9a'
dataset_path = './datasets/{}.txt'.format(data_name)


# regularization parameter
lmb = 1e-3

# number of nodes, size of local data, and dimension of the problem
# according to the paper
N = default_dataset_parameters[data_name]['N']# size of the whole data set
n = default_dataset_parameters[data_name]['n']# number of nodes
m = default_dataset_parameters[data_name]['m']# size of local data set
d = default_dataset_parameters[data_name]['d']# dimension of the problem

batch_size = 512

In [32]:
A, b = read_data(dataset_path=dataset_path, 
                 N=N, n=n, m=m, d=d, lmb=lmb,
                labels=['+1', '-1', '0'])

In [41]:
tensor_x = torch.Tensor(A)
tensor_y = torch.Tensor(b)


my_dataset = TensorDataset(tensor_x,tensor_y) # create your datset
my_dataloader = DataLoader(my_dataset, batch_size=batch_size,shuffle=True) # create your dataloader


In [42]:
for i, (x, y) in zip(range(1000), my_dataloader):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [43]:


class MLP(nn.Module):

    def __init__(self):
        super().__init__()
        self.c_fc    = nn.Linear(d, 1, bias=False)
        #self.c_proj  = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        #self.dropout = nn.Dropout(config.dropout)

    def forward(self, x):
        x = self.c_fc(x)
        return x

In [44]:
model = MLP()

In [45]:
model(tensor_x)

tensor([[ 0.0020],
        [-0.0598],
        [-0.1835],
        ...,
        [-0.3212],
        [ 0.0703],
        [ 0.0518]], grad_fn=<MmBackward0>)

In [ ]:
from model import MLP


# init model loss function and input data

model = Model()
data_loader = ...


In [ ]:
# init the optimizer
optimizer = SophiaG(model.parameters(), lr=2e-4, betas=(0.965, 0.99), rho=0.01, weight_decay=1e-1)

total_bs = len(data_loader)
bs = total_bs * block_size
k = 10
iter_num = -1

# training loop
for epoch in range(epochs):
    for X, Y in data_loader:
        # standard training code
        logits, loss = model(X, Y)
        loss.backward()
        optimizer.step(bs=bs)
        optimizer.zero_grad(set_to_none=True)
        iter_num += 1

        if iter_num % k != k - 1:
            continue
        else:
            # update hessian EMA
            logits, _ = model(X, None)
            samp_dist = torch.distributions.Categorical(logits=logits)
            y_sample = samp_dist.sample()
            loss_sampled = F.cross_entropy(logits.view(-1, logits.size(-1)), y_sample.view(-1), ignore_index=-1)
            loss_sampled.backward()
            optimizer.update_hessian()
            optimizer.zero_grad(set_to_none=True)
            model.zero_grad()